In [1]:
import importlib

import common

importlib.reload(common)

<module 'common' from 'C:\\Users\\j\\IdeaProjects\\fiit_masters_thesis_project\\jupyter\\common.py'>

In [2]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
from common import create_engine, display_all, figsize

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() # converters e.g. for datetime in plots

### Load dataset

In [ ]:
# df = pd.read_csv('final_data.csv')

### Prepare features group 1

### Prepare features group 2

### Prepare features group 3

### Prepare features group 4

### Train test split + prepare labels

---

### Popularity prediction

### Fake new  detection

---

### Results analysis